In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [6]:
W = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [7]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [8]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [9]:
torch.sigmoid(torch.FloatTensor([1]))

tensor([0.7311])

In [10]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [11]:
hypothesis, hypothesis.shape

(tensor([[0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000]], grad_fn=<SigmoidBackward>), torch.Size([6, 1]))

cost
$$cost(W) = - {1 \over m}\sum ylog(H(x)) + (1 - y)(log(1-H(x))$$
$$H(x) = P(x=1;W)$$

In [13]:
# 하나의 엘레멘트만 보면
-(y_train[0] * torch.log(hypothesis[0]) + (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward>)

y_train[0] 이나 (1 - y_train[0]) 둘중에 하나는 0이므로 하나만 반영이 됨

In [15]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))
losses

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)

In [16]:
cost = losses.mean()
cost

tensor(0.6931, grad_fn=<MeanBackward0>)

얘도  함수 지원 F.binary_cross_entropy(hypothesis, y_train)

In [17]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

In [18]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros((1,), requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(' Epoch {:4d} / {} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()))

 Epoch    0 / 1000 Cost: 0.693147
 Epoch  100 / 1000 Cost: 0.134722
 Epoch  200 / 1000 Cost: 0.080643
 Epoch  300 / 1000 Cost: 0.057900
 Epoch  400 / 1000 Cost: 0.045300
 Epoch  500 / 1000 Cost: 0.037261
 Epoch  600 / 1000 Cost: 0.031672
 Epoch  700 / 1000 Cost: 0.027556
 Epoch  800 / 1000 Cost: 0.024394
 Epoch  900 / 1000 Cost: 0.021888
 Epoch 1000 / 1000 Cost: 0.019852


In [26]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
hypothesis[:5]

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)

In [27]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [28]:
prediction[:5] == y_train[:5]

tensor([[True],
        [True],
        [True],
        [True],
        [True]])

## nn.Module을 이용해서 실전

In [29]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(X))

In [30]:
model = BinaryClassifier()

In [32]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print(' Epoch {:4d} / {} Cost: {:.6f} Accuracy {:2.2f}'.format(
        epoch, nb_epochs, cost.item(), accuracy))

 Epoch    0 / 100 Cost: 0.019834 Accuracy 1.00
 Epoch   20 / 100 Cost: 0.019834 Accuracy 1.00
 Epoch   40 / 100 Cost: 0.019834 Accuracy 1.00
 Epoch   60 / 100 Cost: 0.019834 Accuracy 1.00
 Epoch   80 / 100 Cost: 0.019834 Accuracy 1.00
 Epoch  100 / 100 Cost: 0.019834 Accuracy 1.00
